In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Chrome 드라이버 설정 및 URL 설정
url = 'https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlip.do?gmId=G101&gmTs=240128'
driver = webdriver.Chrome()

try:
    # 페이지 열기
    driver.get(url)
    

    # 요소가 나타날 때까지 기다리기 (최대 10초)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#tbd_gmBuySlipList tr'))
    )
    

    # 모든 <tr> 태그 가져오기 (tbody 안의 tr 태그들)
    rows = driver.find_elements(By.CSS_SELECTOR, '#tbd_gmBuySlipList tr')
    

    # 결과 저장용 리스트
    game_data = []

    # 각 행에서 정보 추출
    for index, row in enumerate(rows):
        try:
            
            # 경기 상태 추출 (class "fs11"인 <td> 요소에서 상태 추출)
            game_status = row.find_element(By.CSS_SELECTOR, 'td.fs11').text
            
            
            # '결과발표'인 경기만 처리
            if "결과발표" in game_status:
                # "일반" 텍스트 추출
                try:
                    game_type = row.find_element(By.CSS_SELECTOR, 'span.badge.gray').text  # "일반"만 가져옴
                    
                except Exception:
                    
                    continue
                
                if "일반" in game_type:  # "일반 경기"만 필터링
                    # 홈 팀 이름과 점수 추출
                    home_team = row.find_element(By.CSS_SELECTOR, 'div.cell.tar span').text
                    home_score_text = row.find_element(By.CSS_SELECTOR, 'div.cell.tar strong').text
                    home_score = int(home_score_text.replace("점수\n", "").strip())  # "점수\n" 제거 후 숫자로 변환
                    
                    
                    # 원정 팀 이름과 점수 추출
                    away_team = row.find_element(By.CSS_SELECTOR, 'div.cell.tal span').text
                    away_score_text = row.find_element(By.CSS_SELECTOR, 'div.cell.tal strong').text
                    away_score = int(away_score_text.replace("점수\n", "").strip())  # "점수\n" 제거 후 숫자로 변환
                    
                    
                    # 경기 결과 판단 (승, 무, 패)
                    if home_score > away_score:
                        result = "승"
                    elif home_score == away_score:
                        result = "무"
                    else:
                        result = "패"
                    
                    # 숫자와 소수점만 남기고 나머지 제거하는 함수
                    def extract_only_numbers(text):
                        # 숫자와 소수점(.)만 남기기
                        return ''.join([char for char in text if char.isdigit() or char == '.'])

                    # 승 배당률 추출
                    win_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="1"] span.db').text
                    win_odds = extract_only_numbers(win_odds_text).strip()  # 숫자와 소수점만 추출
                    

                    # 무 배당률 확인 (btnChk이면 배당률, btnNone이면 "-")
                    try:
                        draw_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="2"].btnChk span.db').text
                        draw_odds = extract_only_numbers(draw_odds_text).strip()  # 숫자와 소수점만 추출
                    except:
                        draw_odds = "-"  # 무배당이 없는 경우 "-"
                    

                    # 패 배당률 추출
                    lose_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="3"] span.db').text
                    lose_odds = extract_only_numbers(lose_odds_text).strip()  # 숫자와 소수점만 추출
                    
                    
                    # 데이터 저장
                    game_data.append({
                        "home_team": home_team,
                        "home_score": home_score,
                        "away_team": away_team,
                        "away_score": away_score,
                        "result": result,  # 경기 결과 추가
                        "win_odds": win_odds,
                        "draw_odds": draw_odds,
                        "lose_odds": lose_odds
                    })
        except Exception as e:
            print(f"Error extracting data from row {index + 1}: {e}")

    # 결과 출력
    for idx,game in enumerate(game_data):
        print(f"{idx+1}번째 경기")
        print(f"홈 팀: {game['home_team']}, 홈 팀 점수: {game['home_score']}")
        print(f"원정 팀: {game['away_team']}, 원정 팀 점수: {game['away_score']}")
        print(f"경기 결과: {game['result']}, 승배당: {game['win_odds']}, 무배당: {game['draw_odds']}, 패배당: {game['lose_odds']}")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()

Error extracting data from row 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.fs11"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001054d7634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x00000001054cfe94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000104f3c104 cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000104f7e364 cxxbridge1$string$len + 359360
4   chromedriver                        0x0000000104f748b0 cxxbridge1$string$len + 319756
5   chromedriver                        0x0000000104fb7bd0 cxxbridge1$string$len + 594988
6   chromedriver                        0x0000000104f72f54 cxxbridge1$string$len + 313264
7   chromedriver                        0x000

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Chrome 드라이버 설정 및 URL 설정
url = 'https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlip.do?gmId=G101&gmTs=240128'
driver = webdriver.Chrome()

try:
    # 페이지 열기
    driver.get(url)

    # 요소가 나타날 때까지 기다리기 (최대 10초)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#tbd_gmBuySlipList tr'))
    )

    # 모든 <tr> 태그 가져오기 (tbody 안의 tr 태그들)
    rows = driver.find_elements(By.CSS_SELECTOR, '#tbd_gmBuySlipList tr')

    # 결과 저장용 리스트
    game_data = []

    # 각 행에서 정보 추출
    for index, row in enumerate(rows):
        try:
            # 경기 상태 추출 (class "fs11"인 <td> 요소에서 상태 추출)
            game_status = row.find_element(By.CSS_SELECTOR, 'td.fs11').text

            # '결과발표'인 경기만 처리
            if "결과발표" in game_status:
                # "일반" 텍스트 추출
                try:
                    game_type = row.find_element(By.CSS_SELECTOR, 'span.badge.gray').text  # "일반"만 가져옴
                except Exception:
                    continue

                if "일반" in game_type:  # "일반 경기"만 필터링
                    # 홈 팀과 원정 팀 점수 추출
                    home_score_text = row.find_element(By.CSS_SELECTOR, 'div.cell.tar strong').text
                    home_score = int(home_score_text.replace("점수\n", "").strip())

                    away_score_text = row.find_element(By.CSS_SELECTOR, 'div.cell.tal strong').text
                    away_score = int(away_score_text.replace("점수\n", "").strip())

                    # 경기 결과 판단 (승, 무, 패)
                    if home_score > away_score:
                        result = "승"
                    elif home_score == away_score:
                        result = "무"
                    else:
                        result = "패"

                    # 숫자와 소수점만 남기고 나머지 제거하는 함수
                    def extract_only_numbers(text):
                        return ''.join([char for char in text if char.isdigit() or char == '.'])

                    # 승, 무, 패 배당률 추출
                    win_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="1"] span.db').text
                    win_odds = extract_only_numbers(win_odds_text).strip()

                    try:
                        draw_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="2"].btnChk span.db').text
                        draw_odds = extract_only_numbers(draw_odds_text).strip()
                    except:
                        draw_odds = "-"

                    lose_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="3"] span.db').text
                    lose_odds = extract_only_numbers(lose_odds_text).strip()

                    # 간소화된 데이터 저장
                    game_data.append({
                        "result": result,
                        "win_odds": win_odds,
                        "draw_odds": draw_odds,
                        "lose_odds": lose_odds
                    })
        except Exception as e:
            print(f"Error extracting data from row {index + 1}: {e}")

    # 데이터프레임 생성
    df = pd.DataFrame(game_data)

    # 엑셀 파일로 저장
    df.to_excel("game_results.xlsx", index=False)
    print("엑셀 파일이 성공적으로 저장되었습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()

Error extracting data from row 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.fs11"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102733634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x000000010272be94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000102198104 cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001021da364 cxxbridge1$string$len + 359360
4   chromedriver                        0x00000001021d08b0 cxxbridge1$string$len + 319756
5   chromedriver                        0x0000000102213bd0 cxxbridge1$string$len + 594988
6   chromedriver                        0x00000001021cef54 cxxbridge1$string$len + 313264
7   chromedriver                        0x000

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
# Chrome 드라이버 설정 및 URL 설정
url = 'https://www.betman.co.kr/main/mainPage/gamebuy/closedGameSlip.do?gmId=G101&gmTs=240128'
driver = webdriver.Chrome()

try:
    # 페이지 열기
    driver.get(url)

    # 요소가 나타날 때까지 기다리기 (최대 10초)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#tbd_gmBuySlipList tr'))
    )

    # 모든 <tr> 태그 가져오기 (tbody 안의 tr 태그들)
    rows = driver.find_elements(By.CSS_SELECTOR, '#tbd_gmBuySlipList tr')

    # 결과 저장용 리스트
    game_data = []
    for index, row in enumerate(rows):
        game_type = row.find_element(By.CSS_SELECTOR, 'span.badge.gray').text  # "일반"만 가져옴
           

        if "일반" in game_type:  # "일반 경기"만 필터링
             # 숫자와 소수점만 남기고 나머지 제거하는 함수
                    def extract_only_numbers(text):
                        return ''.join([char for char in text if char.isdigit() or char == '.'])

                    # 승, 무, 패 배당률 추출
                    win_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="1"] span.db').text
                    win_odds = extract_only_numbers(win_odds_text).strip()

                    try:
                        draw_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="2"].btnChk span.db').text
                        draw_odds = extract_only_numbers(draw_odds_text).strip()
                    except:
                        draw_odds = "-"

                    lose_odds_text = row.find_element(By.CSS_SELECTOR, 'button[data-selkey="3"] span.db').text
                    lose_odds = extract_only_numbers(lose_odds_text).strip()

                    # 간소화된 데이터 저장
                    game_data.append({
                        "win_odds": win_odds,
                        "draw_odds": draw_odds,
                        "lose_odds": lose_odds
                    })
                    # 데이터프레임 생성
                    df = pd.DataFrame(game_data)

                    # 엑셀 파일로 저장
                    df.to_excel("trainingData.xlsx", index=False)
                    print("엑셀 파일이 성공적으로 저장되었습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

finally:
    # 브라우저 종료
    driver.quit()


오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[data-selkey="1"] span.db"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104aef634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x0000000104ae7e94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000104554104 cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000104596364 cxxbridge1$string$len + 359360
4   chromedriver                        0x000000010458c8b0 cxxbridge1$string$len + 319756
5   chromedriver                        0x00000001045cfbd0 cxxbridge1$string$len + 594988
6   chromedriver                        0x000000010458af54 cxxbridge1$string$len + 313264
7   chromedriver                        0x000000

Error extracting data from row 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"tr[data-rowindex='0'] td span.db"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100f07634 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x0000000100effe94 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x000000010096c104 cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001009ae364 cxxbridge1$string$len + 359360
4   chromedriver                        0x00000001009a48b0 cxxbridge1$string$len + 319756
5   chromedriver                        0x00000001009e7bd0 cxxbridge1$string$len + 594988
6   chromedriver                        0x00000001009a2f54 cxxbridge1$string$len + 313264
7   chromedriver    